In [66]:
matrix_card = 'G2406344B'
# treat it as ascii
matrix_card_binary = ''.join(format(ord(char), '08b') for char in matrix_card)
# binary to int
matrix_card_int = int(matrix_card_binary, 2)


In [67]:
# protocol parameter
key_length = 1024

# parameter
binary_v = bin(matrix_card_int)[2:]  # 转换为二进制字符串并去掉 '0b' 前缀
missing_bits = key_length - len(binary_v)
if missing_bits > 0:
    padded_binary_v = binary_v + '0' * missing_bits
else:
    padded_binary_v = binary_v
v = int(padded_binary_v, 2)

x1 = 1
x2 = 114514
x3 = 3
iv = b'\x00' * 16

# hash input
file_path = '1.pdf'


In [68]:
#library

import rsa_generation

# SHA3
import hashlib

# AES-128
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad


In [69]:
# Get first 128 bits of shake128 hash digest of the pdf file


def calculate_shake128_128bit_digest(file_path):
    try:
        # open as binary
        with open(file_path, 'rb') as file:
            shake128 = hashlib.shake_128()
            for chunk in iter(lambda: file.read(4096), b""):
                shake128.update(chunk)
            # only obtain first 128 bits
            digest = shake128.digest(16)
            return digest
    except FileNotFoundError:
        print(f"Error: file {file_path} not found.")
        return None
    except Exception as e:
        print(f"Unknown error: {e}.")
        return None
    

In [70]:
# AES-128


def aes_128_cbc_encrypt(plaintext, key, iv):
    # int to byte
    plaintext_bytes = plaintext.to_bytes((plaintext.bit_length() + 7) // 8, byteorder='big')
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ciphertext = cipher.encrypt(pad(plaintext_bytes, AES.block_size))
    # byte to int
    ciphertext_int = int.from_bytes(ciphertext, byteorder='big')
    return ciphertext_int


def aes_128_cbc_decrypt(ciphertext, key, iv):
    # int to byte
    ciphertext_bytes = ciphertext.to_bytes((ciphertext.bit_length() + 7) // 8, byteorder='big')
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted_text = unpad(cipher.decrypt(ciphertext_bytes), AES.block_size)
    # byte to int
    decrypted_text_int = int.from_bytes(decrypted_text, byteorder='big')
    return decrypted_text_int

In [71]:
# ring signature with 3 user


def bitwise_xor(a, b):
    return a ^ b

def ring_signature_compute_result(v, rsa_users, xs, f_add, f_syn_enc, key_sync, iv_sync):
    result = v
    for rsa_user, x in zip(rsa_users, xs):
        result = f_add(result, rsa_user.encrypt(x))
        result = f_syn_enc(result, key_sync, iv_sync)
    return result  # expected to be the same as v


In [72]:
'''main function here'''
users = []
for i in range(1, 4):
    file_path = f'user{i}_keys.txt'
    user = rsa_generation.read_user_from_file(file_path)
    users.append(user)

In [73]:
hash_result = calculate_shake128_128bit_digest(file_path)


In [74]:
xs = [x1, x2, x3]
print(v)
with open('output.txt', 'w') as file:
    file.write(str(v) + '\n')
v_should_be_the_same_as_v = ring_signature_compute_result(v, users, xs, bitwise_xor, aes_128_cbc_encrypt, iv, hash_result)
print(v_should_be_the_same_as_v)
with open('output.txt', 'a') as file:
    file.write(str(v_should_be_the_same_as_v) + '\n')
if v == v_should_be_the_same_as_v:
    message = 'Yes this project is f****** awesome!'
    print(message)
    with open('output.txt', 'a') as file:
        file.write(message + '\n')
else:
    message = 'OMG something is wrong TAT!'
    print(message)
    with open('output.txt', 'a') as file:
        file.write(message + '\n')
        

99991215682665843033985070196595166436044456037235186225817984897421738428148905041599778953441369052565566769336895846599884886454454445402668923576436813917499547546355530983776590577744626677023122268963465745338814663670730182080318619769616369804626532506219915024275340383819518515785846291287173496832
6208131754526899097340384825994320927639483742319227992386194019510068398561816837720300951189997456833360075408939665764864726517863922920040672675233089276484546321944412579907103471167607684545393058765193467167694848836635800296564280526061531103315769621025390256248508470908456775950599044285011524101433587735816680630802656419732596078943923812717395937276984268265059292573714767972379565783868770654488213184721968
OMG something is wrong TAT!
